## **Synthesizing Data from RIRE**

In [0]:
# Ran on Google Colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install SimpleITK
!pip install Pillow
!pip install pypng

     |████████████████████████████████| 42.5MB 104kB/s 
     |████████████████████████████████| 655kB 3.5MB/s 
  Created wheel for pypng: filename=pypng-0.0.20-cp36-none-any.whl size=67161 sha256=11eb5fb96b8b04ad6293e4d11f715dec45dc6a3d71f12bfb055b0008231077f7
  Stored in directory: /root/.cache/pip/wheels/41/6b/ef/0493b536b6d4722c2ae9486691b1d49b922b9877922beeabb3
Successfully built pypng


In [0]:
import os, sys, tarfile

# Modify tgz_path based on where the tar file is uploaded
tgz_path = "./RIREImageDataset.tar"

def extract(tar_url, extract_path='.'):
    print(tar_url)
    tar = tarfile.open(tar_url)
    tar.extractall('.')
    for subTar in tar.getmembers():
      print(subTar.name)
      if '.tgz' in subTar.name:
        subTar = tarfile.open('{}'.format(subTar.name))
        subTar.extractall('./drive/My Drive/CS168 Research/RIREDataset/')
        for item in subTar.getmembers():
          print(item.name)
          if '.tar.gz' in item.name:
            item = tarfile.open('./drive/My Drive/CS168 Research/RIREDataset/{}'.format(item.name))
            item.extractall('./drive/My Drive/CS168 Research/RIREDataset/{}/'.format(subTar.name.split(".tgz")[0].split("RIREDataset/")[1]))
            os.remove(item.name)
        os.remove(subTar.name)

extract(tgz_path)

In [0]:
# %rm -rf RIREDataset/ Only ran this to remove previously failed iterations

In [0]:
import io
import numpy as np
import cv2
import sys
import SimpleITK as sitk
import matplotlib.pyplot as plt

%matplotlib inline

def load_itk(filename):
    # Reads the image using SimpleITK
    itkimage = sitk.ReadImage(filename)

    # Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    ct_scan = sitk.GetArrayFromImage(itkimage)

    # Read the origin of the ct_scan, will be used to convert the coordinates from world to voxel and vice versa.
    origin = np.array(list(reversed(itkimage.GetOrigin())))

    # Read the spacing along each dimension
    spacing = np.array(list(reversed(itkimage.GetSpacing())))

    return itkimage, ct_scan, origin, spacing

# img, ct_scan, origin, spacing = load_itk("./drive/My Drive/CS168 Research/RIREDataset/training_001/mr_T1/training_001_mr_T1.mhd")

# for i in range(len(ct_scan)):
#   plt.imshow(ct_scan[i])


In [0]:
from pathlib import Path
from PIL import Image
import os.path
import imageio
import cv2
import numpy as np

# Replace the paths with the desired paths for writing RIRE images
Path("./drive/My Drive/CS168 Research/RIRECT").mkdir(parents=True, exist_ok=True)
Path("./drive/My Drive/CS168 Research/RIREMRI_T1").mkdir(parents=True, exist_ok=True)
Path("./drive/My Drive/CS168 Research/RIREMRI_T2").mkdir(parents=True, exist_ok=True)
Path("./drive/My Drive/CS168 Research/RIREMRI_PD").mkdir(parents=True, exist_ok=True)

# Replace the path with the path containing the unzipped RIREDataset folder
for path in Path('./drive/My Drive/CS168 Research/RIREDataset').rglob('*.mhd'):
    print(path.absolute())
    img, scan, origin, spacing = load_itk(str(path.absolute()))
    name = path.name.split("/")[-1].split(".mhd")[0]
    print(name)
    if "T1" in name:
      dirPath = "./drive/My Drive/CS168 Research/RIREMRI_T1"
    elif "T2" in name:
      dirPath = "./drive/My Drive/CS168 Research/RIREMRI_T2"
    elif "PD" in name:
      dirPath = "./drive/My Drive/CS168 Research/RIREMRI_PD"
    elif "_ct" in name and "rectified" not in name:
      dirPath = "./drive/My Drive/CS168 Research/RIRECT"
    else:
      dirPath = ""
      continue

    for i in range(len(scan)):
      scan_img = cv2.normalize(scan[i], scan_img, 0, 255, cv2.NORM_MINMAX)

      pathName = "{}/{}_{}.jpg".format(dirPath, name, i)
      imageio.imwrite(pathName, scan_img)


In [0]:
# Replace file paths accordingly
# Download zip file afterwards
!zip -r ./drive/My\ Drive/CS168\ Research/RIREImageDatasetNII.zip ./drive/My\ Drive/CS168\ Research/RIRECT ./drive/My\ Drive/CS168\ Research/RIREMRI_T2 ./drive/My\ Drive/CS168\ Research/RIREMRI_T1 ./drive/My\ Drive/CS168\ Research/RIREMRI_PD

In [0]:
# Only execute this cell if there was a preprocessing error and you want to remove the images
# import shutil

# path1 = "./drive/My Drive/CS168 Research/RIRECT"
# path2 = "./drive/My Drive/CS168 Research/RIREMRI_T2"
# path3 = "./drive/My Drive/CS168 Research/RIREMRI_T1"
# path4 = "./drive/My Drive/CS168 Research/RIREMRI_PD"

# try:
#     shutil.rmtree(path1)
#     shutil.rmtree(path2)
#     shutil.rmtree(path3)
#     shutil.rmtree(path4)
# except OSError as e:
#     print("Error: %s" % (e.strerror))